In [1]:
%load_ext autoreload
%autoreload 2
from vllm import SamplingParams, LLM
from src.text_utils import split_keywords
from src.search_agent_utils import gen_prompt_for_page_choice,gen_prompt_for_keywords,gen_prompt_for_rag
from src.VLLMBot import VLLMBot
import json
import os

/home/setup/miniconda3/envs/llmeval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-06-19 16:31:27,404	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
out_path="processed_wiki/data.jsonl"

In [3]:
model_name = "microsoft/Phi-3-medium-128k-instruct"

llm = LLM(model=model_name, trust_remote_code=True,
          max_model_len=20000
          )

/home/setup/miniconda3/envs/llmeval/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


INFO 06-19 16:31:33 llm_engine.py:100] Initializing an LLM engine (v0.4.2) with config: model='microsoft/Phi-3-medium-128k-instruct', speculative_config=None, tokenizer='microsoft/Phi-3-medium-128k-instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=20000, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=microsoft/Phi-3-medium-128k-instruct)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 06-19 16:31:34 utils.py:660] Found nccl from library /home/setup/.config/vllm/nccl/cu12/libnccl.so.2.18.1
INFO 06-19 16:31:35 selector.py:27] Using FlashAttention-2 backend.
INFO 06-19 16:31:35 weight_utils.py:199] Using model weights format ['*.safetensors']
INFO 06-19 16:31:40 model_runner.py:175] Loading model weights took 26.1473 GB
INFO 06-19 16:31:43 gpu_executor.py:114] # GPU blocks: 13470, # CPU blocks: 1310
INFO 06-19 16:31:44 model_runner.py:937] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 06-19 16:31:44 model_runner.py:941] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 06-19 16:31:48 model_runner.py:1017] Graph capturing 

In [4]:
bot=VLLMBot(llm)

In [5]:
import wikipedia
wikipedia.set_lang("ja")

In [7]:

q="半沢直樹の有名なセリフは?"


In [18]:
def rag_ask(q,bot,max_keys=3,verbose=False):
    prompt=gen_prompt_for_keywords(q)

    if verbose:
        print("-------------------------------")
        print(f"Q1.\n{prompt}")

    str_keywords=bot.ask(prompt,max_tokens=1000)

    if verbose:
        print(f"A1.\n{str_keywords}")

    keywords=split_keywords(str_keywords)

    max_keys=min(len(keywords),max_keys)
    wiki_hit_titles=[]
    for i in range(max_keys):
        wiki_hit_titles.extend(wikipedia.search(keywords[i]))

    if verbose:
        print("-------------------------------")
        print("hit titles: ",wiki_hit_titles)


    page_choice_prompt=gen_prompt_for_page_choice(q,wiki_hit_titles)
    if verbose:
        print("-------------------------------")
        print(f"Q2.\n{page_choice_prompt}")
    selected_page_title=bot.ask(page_choice_prompt)

    if verbose:
        print("-------------------------------")
        print(f"A2.\n{selected_page_title}")

    page=wikipedia.page(selected_page_title)

    rag_prompt=gen_prompt_for_rag(q,page.content,max_content_length=10000)

    if verbose:
        print("-------------------------------")
        print(f"Q3.\n{rag_prompt}")

    ans=bot.ask(rag_prompt)
    if verbose:
        print("-------------------------------")
        print(f"A3.\n{ans}")
    return ans

In [19]:
a=rag_ask(q,bot,max_keys=2,
          verbose=True)
a

-------------------------------
Q1.
<|user|>
次の質問に回答するために検索用の主要なキーワードをカンマ区切りのリストで出力しなさい｡
-単語以外は出力しないこと

#例
#質問
日本の高齢化社会について考える上で大切な論点は?
#キーワード
日本,高齢化,社会

#質問
半沢直樹の有名なセリフは?
#キーワード
<|end|>
<|assistant|>


Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  2.80it/s]


A1.
半沢直樹,有名な、セリフ
-------------------------------
hit titles:  ['半沢直樹', '堺雅人', '片岡愛之助 (6代目)', '池井戸潤', '半沢直樹シリーズ', '福澤克雄', '服部隆之', '筒井道隆', '土下座', '高村直樹', 'セレブリティ', '恵那市', '高名な依頼人', '物理学者の一覧', 'カメラマン', 'ギリシア神話の固有名詞一覧', 'はくちょう座', '大理石', '数学者', '三浦市']
-------------------------------
Q2.
<|user|>
次の質問に回答するために最も適した単語を､#キーワードから一つだけ選びなさい｡
-もし可能なら､質問文から単語を選ぶこと
-質問文と関係性の高い単語を選ぶこと

#例
#質問
日本の総理大臣は?
#キーワード
日本,総理大臣,趣味,遊び
#出力
総理大臣

#質問
半沢直樹の有名なセリフは?
#キーワード
半沢直樹,堺雅人,片岡愛之助 (6代目),池井戸潤,半沢直樹シリーズ,福澤克雄,服部隆之,筒井道隆,土下座,高村直樹,セレブリティ,恵那市,高名な依頼人,物理学者の一覧,カメラマン,ギリシア神話の固有名詞一覧,はくちょう座,大理石,数学者,三浦市
#出力
<|end|>
<|assistant|>


Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  3.38it/s]


-------------------------------
A2.
半沢直樹シリーズ
-------------------------------
Q3.
<|user|>
#情報をもとに､質問に回答しなさい｡ただし､#情報は間違っているケースも多いので､その場合は#情報は参考にせず回答しなさい｡
#情報
半沢直樹シリーズ（はんざわなおきシリーズ）は、池井戸潤による企業エンターテインメント小説シリーズである。
当初は「オレバブシリーズ」と呼ばれていたが、2013年にTBS系『日曜劇場』枠でのテレビドラマ化を機にシリーズ名を変更した。


== 沿革 ==
バブル末期に大手都市銀行「産業中央銀行」に入行した半沢直樹が、産業中央銀行と「東京第一銀行」が合併して誕生した「東京中央銀行」において銀行内外の人間や組織による数々の圧力や逆境と戦う姿を描くシリーズ小説。合併後の銀行が舞台で、合併前の東京第一銀行を舞台とした「花咲舞シリーズ」から、合併前後の物語に直接受け継がれるようになっている。
池井戸自身もバブル期に三菱銀行（現・三菱UFJ銀行）に入行した銀行員であったが、自身の経験はあまり書かれてはいない。自分がわからない人物だと書けないと思うようになったことから、池井戸に一番近い年代の主人公にしたという。
従来の企業小説では銀行の悲惨さや陰惨さを暴露的に訴えるような暗い作品が多かったが、それらをひっくり返して「銀行の中で人が生き生き動く活劇」を作る過程で出来上がったのが、ズバズバ物を言うが小狡い一面も持つ「清濁併せ飲むヒーロー・半沢直樹」であり「サラリーマンチャンバラ劇」の趣となっていると池井戸は語っている。第1作を単行本化するに際して、読者にはエンターテインメントとして読んでほしいという思いから、企業小説風のネーミングではなく敢えて『オレたちバブル入行組』という物語風のタイトルをつけた。
第1作『オレたちバブル入行組』・第2作『オレたち花のバブル組』は『別冊文藝春秋』での連載を経て、文藝春秋より単行本化され、その後文春文庫および講談社文庫にて文庫化された。第3作『ロスジェネの逆襲』・第4作『銀翼のイカロス』は『週刊ダイヤモンド』での連載を経て、ダイヤモンド社より単行本化され、その後文春文庫および講談社文庫にて文庫化された。第5作『アルルカンと道化師』は書き下ろしで講

Processed prompts: 100%|██████████| 1/1 [00:05<00:00,  5.80s/it]

-------------------------------
A3.
半沢直樹の有名なセリフは、「'''やられたらやり返す!倍返し!!'''」です。このセリフは、彼が巨額の融資ミスに対する浅野匡の非道な行動に対して、彼が宣言したもので、彼の典型的な性格と銀行界における正義感を象徴するものとして広く知られています。


"半沢直樹の有名なセリフは、「'''やられたらやり返す!倍返し!!'''」です。このセリフは、彼が巨額の融資ミスに対する浅野匡の非道な行動に対して、彼が宣言したもので、彼の典型的な性格と銀行界における正義感を象徴するものとして広く知られています。"

In [50]:
prompt=gen_prompt_for_keywords(q,)
str_keywords=bot.ask(prompt)
keywords=split_keywords(str_keywords)


Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.76it/s]


In [52]:
wiki_hit_titles

['半沢',
 '半沢直樹',
 '堺雅人',
 '半沢直樹シリーズ',
 'FreeTEMPO',
 '池井戸潤',
 '片岡愛之助 (6代目)',
 '福澤克雄',
 '筒井道隆',
 '服部隆之']

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  5.32it/s]


In [42]:
page=wikipedia.page(selected_page_title)

In [47]:
rag_prompt=gen_prompt_for_rag(q,page.content)
rag_prompt

'<|user|>\n参考情報をもとに､質問に回答しなさい｡\n#参考情報\n『半沢直樹』（はんざわ なおき）は、TBS系列「日曜劇場」枠で放送された、池井戸潤の小説「半沢直樹シリーズ」を原作としたテレビドラマである。主演は堺雅人。\n2013年7月7日から9月22日まで、『オレたちバブル入行組』と『オレたち花のバブル組』をベースとした前後編二部構成・全10話が放送された。\n2020年1月3日23時15分から翌0時45分までスピンオフドラマ『半沢直樹II・エピソードゼロ〜狙われた半沢直樹のパスワード〜』（はんざわなおきツー・エピソードゼロ ねらわれたはんざわなおきのパスワード）が放送された。主演は吉沢亮。\n2020年7月19日から9月27日まで、同じく「日曜劇場」枠で『ロスジェネの逆襲』と『銀翼のイカロス』をベースとした全10話の続編が放送された。\nキャッチコピーは「やられたらやり返す、倍返しだ!!」、「クソ上司め、覚えていやがれ!!」（2013年版）。\n\n\n== 概要 ==\n共通の主人公が登場する原作においては各巻ごとに表題が異なるが、「テレビドラマの表題が異なるのは好ましくない」として表題を統一している。また、「主人公の生き様に焦点を当てる」という思いから、主人公の名前を表題としている。\n\n\n=== 2013年版 ===\n本作は、銀行を巡る不正を描くフィクション作品であり、一般的には視聴率を取りにくい「経済ドラマ」というジャンルである。制作サイドは、主な視聴層を30代以上の男性と想定し、12 - 13%から視聴率が徐々に上がり最終話で20%、平均視聴率15%くらいが取れれば良いと考えていたという。\n銀行という男性の世界を舞台にしており、「中年ばかりで、視聴率を取れそうな若いキャラクターが少ない」「女性の登場人物も少ない」「目立った恋愛シーンもない」「主題歌や挿入歌といったテーマソングもない」という「ないない尽くし」の作品であり、それまでのテレビドラマ業界の常識に外れるものであった。\nしかし、実力派俳優陣による骨太の演技、わかりやすい主人公と敵、チャンバラを思わせるアクションシーン、リアルかつ時代劇調の脚本・演出、現代社会のサラリーマンのテーマでもある「本当は上司に言いたい!」といった心理を如実に映し出した内容が支持され、第1話から19.

In [48]:
ans=bot.ask(rag_prompt)

Processed prompts: 100%|██████████| 1/1 [00:09<00:00,  9.18s/it]


In [49]:
ans

'半沢直樹は、TBS系列「日曜劇場」枠で放送されたテレビドラマ「半沢直樹シリーズ」の主人公であり、池井戸潤の小説「半沢直樹シリーズ」を原作としたドラマである。彼は銀行マンとして有能であり、「上を目指す」と公言している。彼の動機は、彼が中学生の頃に母親が死亡し、父親が経営していた工場が傾いたことで銀行に融資を引き揚げられ、父親が自殺した過去があるためである。半沢が入行した産業中央銀行(旧S)は後に巨額の不良債権を抱えたことで、東京第一銀行(旧T)との合併を実行し、世界第三位のメガバンク・東京中央銀行となる。しかし、上層部では旧S派と旧T派の醜い派閥争いが繰り広げられていた。'